In [ ]:
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import cvxopt
data=pd.read_csv("KDD10.csv")
data=data[data.columns[1:]]

data['outcome']=data['outcome'].replace(r'\.','',regex=True).str.strip()
y=data[["outcome"]]

def LabelEncoding(df):
    for col in df.columns:
        if df[col].dtype == 'object':
                label_encoder = LabelEncoder()
                df[col] = label_encoder.fit_transform(df[col])
X=data.drop("outcome",axis=1)
LabelEncoding(X)
LabelEncoding(y)
y=y.values
y


C:\Users\Kartik\AppData\Local\Temp\ipykernel_29360\3669406887.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = label_encoder.fit_transform(df[col])


array([[11],
       [11],
       [11],
       ...,
       [11],
       [11],
       [11]])

In [ ]:
pca = PCA(n_components=40,svd_solver="covariance_eigh")
pca.fit(X)
X=pca.transform(X)
X.shape
pca.explained_variance_

array([9.76575015e+11, 1.09164184e+09, 5.02196226e+05, 1.07664693e+05,
       9.57080427e+03, 2.15483161e+03, 8.53681451e+02, 3.03660397e+01,
       7.23439534e+00, 1.72168538e+00, 5.97389690e-01, 2.13030193e-01,
       5.35814534e-02, 2.20658542e-02, 2.05122126e-02, 1.66096621e-02,
       1.63224881e-02, 1.01520782e-02, 9.24889742e-03, 7.71645349e-03])

In [4]:

# class SVM:
#     def __init__(self,learning_rate=0.01,lambda_param=0.01,n_iters=1):
#         self.learning_rate=learning_rate
#         self.lambda_param=lambda_param
#         self.n_iters=n_iters
#         self.model={}
#     def fit(self,X,y):
#         n_samples,n_features=X.shape
#         classes=np.unique(y)
#         for cls in classes:
#             y_=np.where(y==cls,1,-1).astype(int)
#             w=np.zeros(n_features)
#             b=0
#             for _ in range(self.n_iters):
#                 for indxs, x_i in enumerate(X):
#                     condition=(y_[indxs]*(np.dot(w,x_i)-b)) >=1
#                     if condition:
#                         w-=self.learning_rate*(2*self.lambda_param*w)
#                     else :
#                         w-=self.learning_rate*((2*self.lambda_param*w)-x_i*y_[indxs])
#                         b-=self.learning_rate*y_[indxs]
            
#             self.model[cls]=(w,b)
#     def predict(self,X):
#         decision_values={}
        
#         for cls,(w,b) in self.model.items():
#             decision_values[cls]=np.dot(X,w)-b
#         decision_values_array = np.array(list(decision_values.values()))   
#         max_indices = np.argmax(decision_values_array, axis=0)  
#         predicted_classes = [list(self.model.keys())[idx] for idx in max_indices]

#         return predicted_classes
            
     

In [ ]:


def linear(x, z):
    return np.dot(x, z.T)

class SVM:
    def __init__(self, kernel=linear, C=1):
        self.kernel = kernel
        self.C = C

    def fit(self, X, y):
        self.y = y
        self.X = X
        m, n = X.shape

        self.K = np.zeros((m, m))
        for i in range(m):
            self.K[i, :] = self.kernel(X[i, np.newaxis], self.X)
        P = cvxopt.matrix(np.outer(y, y) * self.K)
        q = cvxopt.matrix(-np.ones((m, 1)))
        G = cvxopt.matrix(np.vstack((np.eye(m) * -1, np.eye(m))))
        h = cvxopt.matrix(np.hstack((np.zeros(m), np.ones(m) * self.C)))
        A = cvxopt.matrix(y, (1, m), "d")
        b = cvxopt.matrix(np.zeros(1))
        cvxopt.solvers.options["show_progress"] = False
        sol = cvxopt.solvers.qp(P, q, G, h, A, b)
        self.alphas = np.array(sol["x"])

    def predict(self, X):
        y_predict = np.zeros((X.shape[0]))
        sv = self.get_parameters(self.alphas)

        for i in range(X.shape[0]):
            y_predict[i] = np.sum(
                self.alphas[sv]
                * self.y[sv, np.newaxis]
                * self.kernel(X[i], self.X[sv])[:, np.newaxis]
            )

        return np.sign(y_predict + self.b)

    def get_parameters(self, alphas):
        threshold = 1e-5

        sv = ((alphas > threshold) * (alphas < self.C)).flatten()
        self.w = np.dot(X[sv].T, alphas[sv] * self.y[sv, np.newaxis])
        self.b = np.mean(
            self.y[sv, np.newaxis]
            - self.alphas[sv] * self.y[sv, np.newaxis] * self.K[sv, sv][:, np.newaxis]
        )
        return sv


In [ ]:

y=y.ravel()
split=int(0.1*len(X))
X_train=X[:split]
X_test=X[split:]
y_train=y[:split]
y_test=y[split:]

from sklearn.svm import SVC
svm_model = SVC(kernel='linear')
svm_model.fit(X_train,y_train)
y_predict=svm_model.predict(X_train)



In [ ]:
y.shape

(494021,)